In [20]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
port_stemmer = PorterStemmer()

from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from keras import layers, models
from keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Howlingwolfs\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Howlingwolfs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Howlingwolfs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [21]:
data = pd.read_csv('Train.csv')
train = data.copy()
train.drop_duplicates(inplace=True)
train['tokens'] = train['text'].apply(word_tokenize)

In [22]:
imdb = models.load_model('Imdb.keras')

In [23]:
def preprocessing(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z]", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'[^\w\s,.!?]', '', text)  # removes most emojis & symbols
    text = text.split()
    text = [port_stemmer.stem(word) for word in text if not word in stop_words]
    return text

In [24]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(train['tokens'])

In [25]:
r = "This isn't just a reboot-it's a full-on resurrection of everything we love about Superman! James Gunn nails the tone: emotional, bold, and full of hope. David Corenswet owns the cape-he's charming, powerful, and finally gives us a Superman we can believe in. Rachel Brosnahan kills it as Lois Lane with wit and fire. The action is epic, the visuals are stunning, and the story hits the heart. Sure, the CGI finale gets a bit wild, but who cares? This movie flies high and brings the DC Universe back to life. A must-watch for any superhero fan! Can't wait to see where this universe goes next-this is just the beginning"

In [32]:
def process_review(review, max_length=180):
    review_processed = preprocessing(review)
    review_token = word_tokenize(review)
    review_sequences = tokenizer.texts_to_sequences([review_token])
    review_final = pad_sequences(review_sequences, maxlen=max_length, padding='post', truncating='post')
    print(review_final.shape)
    return review_final

In [33]:
def predict(review):
    predicting = imdb.predict(review, verbose=1)
    predicted_classes = (predicting > 0.5).astype("int32")
    predicted_classes.resize(1)
    ans = int(predicted_classes)
    if ans == 1:
        print('\n===< The review is positive >===\n')
    elif ans == 0:
        print('\n===< The review is negative >===\n')

In [34]:
review = process_review(r)
predict(review)

(1, 180)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step

===< The review is positive >===



C:\Users\Howlingwolfs\AppData\Local\Temp\ipykernel_12616\533393286.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ans = int(predicted_classes)
